이번 세션은 케라스로 아~주 간단하게 NLP를 위한 파이프라인에 대해 소개한다.

# 1. 전처리(Preprocessing)
---

## Tokenizer() 

학습 데이터(문장)을 통해 단어 집합을 생성하고, 임의의 문장을 정수로 인코딩하는 과정에 사용된다.


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
train_text = "The earth is an awesome place live"

# 단어 집합 생성
tokenizer.fit_on_texts([train_text])

# 정수 인코딩
text_for_test = "The earth is an great place live"
sequences = tokenizer.texts_to_sequences([text_for_test])[0]

print("정수 인코딩 : ",sequences)
print("단어 집합 : ",tokenizer.word_index)

정수 인코딩 :  [1, 2, 3, 4, 6, 7]
단어 집합 :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


train_text에는 great이란 단어가 없었기 때문에, text_for_test를 인코딩하면 great은 인코딩되지 않은 모습을 볼 수 있다.

## pad_sequence()

각각의 학습 데이터들의 길이가 모두 일정하면 좋지만, 그렇지 않은 경우가 대부분이다. pad_sequence()는 zero padding을 사용해서 문장의 길이를 맞춰준다. 정해준 길이 (maxlen)보다 크면 자르고 작으면 zero패딩을 한다.


In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

arr = [[1,2,3],[4,5,6,7],[8,9]]

# zero padding을 전에 넣기
print("padding='pre'")
print(pad_sequences(arr,maxlen=3,padding='pre'))
print()
# 후에 넣기
print("padding='post'")
print(pad_sequences(arr,maxlen=3,padding='post'))


padding='pre'
[[1 2 3]
 [5 6 7]
 [0 8 9]]

padding='post'
[[1 2 3]
 [5 6 7]
 [8 9 0]]


<br/><br/>

# 2. 워드 임베딩(Word Embedding)
---

원-핫 인코딩을 사용하면 단어가 많을 땐 데이터가 차원이 너무 큰 sparse 데이터가 되어버린다. 그리고 원-핫 벡터는 단어 벡터간의 유의미한 유사도를 구할 수 없다. 워드 임베딩을 사용해 얻은 임베딩 벡터는 좀 더 저차원 벡터를 갖게 되고, 모든 원소 값이 실수이다. 

워드 임베딩으로 단어를 밀집 벡터(dense vector) 혹은 임베딩 벡터(embedding vector)로 만든다. 차원은 주로 256,512,1024를 갖는다고 한다. 초기 값은 랜덤 값을 갖지만, 가중치가 학습되는 방식처럼 값이 학습되면서 변경된다고 한다.

## Embedding()

단어를 밀집 벡터로 만들고 임베딩 층(embedding layer)를 만드는 역할을 한다. tokenizer를 통해 정수 인코딩된 단어들을 입력 받아서 임베딩을 수행한다. 즉, 단어 집합으로 정수 인코딩된 문장의 **함축적인 의미를 갖는 저차원의 벡터**를 만든다고 생각하면 된다.

간단한 예시를 보자.

#### 1. 토큰화
tokenized_text = [['Hope', 'to', 'see', 'you', 'soon'], ['Nice', 'to', 'see', 'you', 'again']]

#### 2. 각 단어에 대한 정수 인코딩
encoded_text = [[0, 1, 2, 3, 4],[5, 1, 2, 3, 6]]

#### 3. 위 정수 인코딩 데이터가 아래의 임베딩 층의 입력이 된다.
vocab_size = 7 # 단어 집합의 크기(총 단어의 개수)<br/>
embedding_dim = 2 # 임베딩 벡터의 출력 차원<br/>
Embedding(vocab_size, embedding_dim, input_length=5)<br/>

#### 각 정수는 아래의 테이블의 인덱스로 사용되며 Embedding()은 각 단어마다 임베딩 벡터를 리턴한다.
|   index    | embedding  |
|:--:|:--:|
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |

#### 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.

<br/><br/>

# 3. 모델링(Modeling)
---

## Sequential()

여태까지 간단한 예제들을 살펴볼 때, 항상 model = Sequential() 객체를 생성해서 model.add(...)를 사용해 층을 추가했다. 이건 되게 직관적으로 모델링을 표현할 수 있기 때문에 좋은 방법인 것 같다. 나중에 pytorch에 가서도 이런 식으로 모델링을 해봐야지.

간단하게 FC-layer들로 하나의 클래스에 대한 확률을 예측하는 모델을 만들어보자.

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(8,input_dim=3,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

## summary()

모델의 정보를 요약해서 보여준다.

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 32        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


<br/><br/>

# 4. 컴파일(Compile)과 훈련(Training)
---

## compile() 
모델을 기계가 이해할 수 있도록 컴파일한다. 옵티마이저,손실함수,메트릭을 선택한다.

RNN을 이용한 이진 분류를 하는 예시 코드를 통해 봐보자.

In [23]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

vocab_size = 10000
embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


## fit()
모델에 데이터를 넣고 학습할 때 사용한다. 사이킷런을 사용할 때와 비슷하다. 

```python

model.fit(X_train, y_train, epochs=10, batch_size=32)

```

#### 검증 데이터셋을 활용하고 싶을 땐 2가지 방법이 있다.

1. 검증 데이터 셋을 직접 만들었을 때
```python
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data(X_val, y_val))
```
2. 학습 데이터 셋에서 검증 데이터를 뽑아서 쓸 때
```python
# 훈련 데이터의 20%를 검증 데이터로 사용.
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_split=0.2))
```

검증 데이터 셋을 활용하게 되면, 매 에포크마다 검증 데이터의 정확도나 오차가 함께 출력된다.

#### verbose는 학습 중 출력되는 문구 옵션이라고 한다.

- 0 : 아무 것도 출력하지 않는다.
- 1 : 훈련의 진행도를 보여주는 진행 막대를 보여준다.

    ```python
    # verbose = 1일 경우.
    Epoch 88/100
    7/7 [==============================] - 0s 143us/step - loss: 0.1029 - acc: 1.0000
    ```

- 2 : 미니 배치마다 손실 정보를 출력한다.
    
    ```python
    # verbose = 2일 경우.
    Epoch 88/100
     - 0s - loss: 0.1475 - acc: 1.0000
    ```
<br/><br/>

# 5. 평가(Evaluation)와 예측(Prediction)
---

## evaluate()
테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가한다.

```python

# 위의 fit() 코드의 연장선상인 코드
model.evaluate(X_test, y_test, batch_size=32)

```

## predict() : 임의의 입력에 대한 모델의 출력값을 확인합니다.


```python

# 위의 fit() 코드의 연장선상인 코드
model.predict(X_input, batch_size=32)

```

<br/><br/>
# 6. 모델의 저장(Save)과 로드(Load)
---

학습된 모델의 파라미터들을 저장하고 불러온다.

## save() 
인공 신경망 모델을 hdf5 파일에 저장한다.
```python

model.save("model_name.h5")

```


## load_model() 

저장해둔 모델을 불러온다.

```python

from tensorflow.keras.models import load_model
model = load_model("model_name.h5")
```

